In [5]:
import sys
import torch
import numpy as np
import DataTools as Tools
import matplotlib.pyplot as plt
from Datasets import SimulateTrain
from network import CADET
from torch.utils.data import DataLoader
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [6]:
datasets = {
    "JPEGImages" : "*.jpg"
}
dirpath = "F:\\Program File\\DataSpell\\Projetcs\\ImageDenoising\\ConvolutionalPoissonNoise\\dsProject\\TestMethods\\data"

A = Tools.get_gt_image(dirpath, datasets)
train_paths = np.array(sorted(A))
X = SimulateTrain(im_list=train_paths, length=64, patch_size=128, peak=20)

In [7]:
gt, noisy = X.__getitem__(1000)
gt.shape, noisy.shape

(torch.Size([1, 128, 128]), torch.Size([1, 128, 128]))

In [8]:
# tensor.cuda() ----> numpy()
gt_np = gt.cpu().numpy()
noisy_np = noisy.cpu().numpy()

In [9]:
# (1, h, w) ---> (h, w)
gt_np = gt_np.transpose((1, 2, 0)).squeeze()
noisy_np = noisy_np.transpose((1, 2, 0)).squeeze()

In [10]:
plt.subplot(131), plt.imshow(gt_np, 'gray'), plt.title("Original Image")
plt.subplot(132), plt.imshow(noisy_np, 'gray'), plt.title("Noisy Image")

(<AxesSubplot:title={'center':'Noisy Image'}>,
 Text(0.5, 1.0, 'Noisy Image'))

## 加载模型参数

In [11]:
model = CADET(in_channels=1, wf=63)
checkpoint = torch.load("F:\\Program File\\DataSpell\\Projetcs\\ImageDenoising\\ConvolutionalPoissonNoise\\dsProject\\Models\\model40.pth")

In [12]:
state_one = model.state_dict()
for name, value in checkpoint["model_state_dict"].items():
    state_one[name] = value
# 仅添加网络的参数即可
model.load_state_dict(state_one)

<All keys matched successfully>

In [13]:
denoise = model(noisy)

In [14]:
denoise_np = denoise.detach().cpu().numpy()

In [15]:
denoise_np = denoise_np.transpose((1, 2, 0)).squeeze()

In [16]:
gt_np = gt_np + 0.5
noisy_np = noisy_np + 0.5
denoise_np = denoise_np + 0.5

In [18]:
plt.subplot(131), plt.imshow(gt_np, 'gray'), plt.title("Original Image")
plt.subplot(132), plt.imshow(noisy_np, 'gray'), plt.title("Noisy Image")
plt.subplot(133), plt.imshow(denoise_np, 'gray'), plt.title("denoise")

(<AxesSubplot:title={'center':'denoise'}>,
 Text(0.5, 1.0, 'denoise'))

In [19]:
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

In [20]:
psnr_noisy = psnr(image_true=(gt_np * 255).astype(np.uint8), image_test=(noisy_np * 255).astype(np.uint8), data_range=255)
psnr_denoise = psnr(image_true=(gt_np * 255).astype(np.uint8), image_test=(denoise_np * 255).astype(np.uint8), data_range=255)

In [21]:
ssim_noisy = ssim(im1=(gt_np * 255).astype(np.uint8), im2=(noisy_np * 255).astype(np.uint8),
                  data_range=255, multichannel=False)
ssim_denoise =ssim(im1=(gt_np * 255).astype(np.uint8), im2=(denoise_np * 255).astype(np.uint8),
                    data_range=255, multichannel=False)

In [22]:
print("Noisy and Gound truth: psnr:  {0}  ,  ssim:  {1}".format(psnr_noisy, ssim_noisy))
print("Denoise and Gound truth: psnr:  {0}  ,  ssim:  {1}".format(psnr_denoise, ssim_denoise))

Noisy and Gound truth: psnr:  13.152121476471738  ,  ssim:  0.348160773538488
Denoise and Gound truth: psnr:  11.83349583043116  ,  ssim:  0.17153024381077195


In [23]:
import torch.nn as nn
mse = nn.MSELoss()

In [24]:
mse(gt.cpu(), denoise.cpu())

tensor(0.0345, grad_fn=<MseLossBackward0>)

In [25]:
model.features()

AttributeError: 'CADET' object has no attribute 'features'

In [26]:
print(model)

CADET(
  (oconv1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1): Conv2d(1, 63, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU(inplace=True)
  (vgg): Sequential(
    (0): Conv2d(63, 63, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
  )
  (oconv2): Conv2d(63, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
